In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("simple app").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/26 17:22:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.sql("SELECT 1 AS x").show()

+---+
|  x|
+---+
|  1|
+---+

